# Coletando dados de saúde suplementar da ANS
Esta rotina de coleta foi feita para ser compatível com ambientes de computação distribuída, para chegar neste objetivo, será usado:

- `PySpark` para rotina de coleta e armazenamento de dados
- Oracle Cloud Infrastructure (OCI), modalidade gratuita para Data Warehousing

Inicialmente, este projeto foi pensado para executar nas instâncias de computação distribuída gratuitas do **Databicks**, mas isto acabou sendo descartado por causa do método de autenticação escolhido para acessar o banco de dados. Para manter o maior nível de segurança, esta rotina será executada localmente.




### Dependências

Vamos instalar um [pacote desenvolvido por mim](https://pypi.org/project/ftp-download/) para superar o desafio de realizar múltiplos downloads de arquivos em servidores, e na sequência vamos instalar as demais dependências, incluindo o PySpark, que será usado aqui para transportar os dados para o banco de dados remoto.

In [1]:
%python -m pip install --upgrade -q pip
%pip install -qr requirements.txt

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [ ]:
import sys
import re
import ftp_download as ftpd
from os import path, listdir, makedirs
import logging
from ftplib import FTP
from getpass import getpass
from shutil import unpack_archive
from pyspark.dbutils import DBUtils
from pyspark.sql import SparkSession
from pyspark.sql.types import *


# sys.path.append('/dbfs/FileStore/tables/collectutils')
# import collectutils

s = SparkSession.builder.appName("dbInteract").getOrCreate()

In [ ]:
TNS_STR = "(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1522)(host=adb.sa-saopaulo-1.oraclecloud.com))(connect_data=(service_name=g8c67f84bc4a850_demodb_high.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes)))"

DRIVER = "oracle.jdbc.driver.OracleDriver"

USR = "ADMIN"
# PWD = getpass("Insira a senha de acesso do Administrador do banco de dados: ")
# URL = f"jdbc:oracle:thin:{USR}/{PWD}//adb.sa-saopaulo-1.oraclecloud.com:1522/g8c67f84bc4a850_demodb_high.adb.oraclecloud.com"
URL = f"jdbc:oracle:thin:@{TNS_STR}"


### Preparando a rotina de coleta

Nosso objetivo é conectar à um servidor FTP e baixar uma quantidade considerável de arquivos, o design da rotina abaixo leva em consideração **nossas limitações, que são**:

1. Os arquivos são `.csv` compactados dentro de arquivos `.zip`;
2. Existem dois tipos de tabelas: 
  a. *Detalhada*, e
  b. *Consolidada*, estas devem compor tabelas diferentes do nosso banco de dados;
3. Todos estes arquivos estão separados em pastas, uma para cada estado

A **estratégia adotada** consiste em:

1. Fazer múltiplos downloads concorrentes usando computação assíncrona;
2. Descompactar e organizar os arquivos localmente
3. Usar o pyspark para ler e guardar os dados no banco de dados;

In [ ]:
FTP_SERVER = "ftp.dadosabertos.ans.gov.br"
ROOT_FOLDER_SRC = "/FTP/PDA/TISS/HOSPITALAR/2019/"

ftp = FTP(FTP_SERVER)
ftp.login()
remote_paths = ftp.nlst(ROOT_FOLDER_SRC)

print(remote_paths[:3])

Já sabemos que os arquivos no servidor possuem nomes regulares, eles possuem sufixo "DET" nos arquivos com a tabela detalhada, e "CONS" nas tabelas consolidadas. Sabemos também que todos os arquivos estão compactados com extensão `.zip`.

Para superar estes dois desafios, vamos criar uma função que identifica o tipo de tabela pelo nome do arquivo e extrai os conteúdos em um diretório em comum, de modo que todas as tabelas consolidadas estejam armazenadas em uma pasta, e todas as tabelas detalhadas estejam armazenadas em uma outra pasta.

In [ ]:
def extract_and_organize(search_dir: str, find_patterns=["CONS", "DET"]):
    def filter_by_pattern(pattern, elements):
        matches = re.compile(pattern, re.IGNORECASE)
        return list(filter(matches.search, elements))

    filepaths = [
        f for f in listdir(search_dir) 
        if path.isfile(path.join(search_dir, f))
    ]
    listings = {i:filter_by_pattern(i, filepaths) for i in find_patterns}

    for pattern in find_patterns:
        destination = path.join(search_dir, pattern)

        if not path.exists(destination):
            makedirs(destination)

        for f in listings[pattern]:
            origin = path.join(search_dir, f)
            unpack_archive(origin, destination, "zip")

A próxima etapa vai envolver o uso de um pacote de minha autoria `ftp_download`, para saber mais sobre o projeto, visite [https://pypi.org/project/ftp-download/](https://pypi.org/project/ftp-download/). A rotina na célula abaixo segue três etapas:

1. **Download** dos arquivos para o ambiente local com `ftp_download`
2. **Descompactação e organização** dos arquivos baixados em pastas usando a rotina desenvolvida na célula anterior
3. **Movimentação dos arquivos** da unidade local para a base de dados NoSQL do databricks `dbfs`, onde o pyspark tem acesso

In [ ]:
# 1
ftpd.Conf.verbose = False
# exibir log apenas com avisos e erros
ftpd.timings.log.handler.setLevel(logging.WARNING)
ftpd.timings.log.logger.setLevel(logging.WARNING)

for i, rp in enumerate(remote_paths):
    print(f"Progresso: {i/len(remote_paths)*100:.2f}%", end="\r")
    ftpd.from_folder(ftp, remote_path=rp)
print("=== Concluído! ===")

with open(ftpd.prefs.LOG_FILE) as logfile:
    logfile_contents = logfile.read()
    print(logfile_contents)

In [ ]:
# 2
download_place = ftpd.Conf.download_folder
extract_and_organize(search_dir=download_place)

In [ ]:
# 3
for subfolder in ["CONS", "DET"]:
    dbutils.fs.mv(
        f"file:{path.join(download_place, subfolder)}", 
        f"dbfs:{path.join('/ans/hosp/2019/', subfolder)}",
        recurse=True)

display(dbutils.fs.ls("dbfs:/ans/hosp/2019/"))

# Preparando a rotina de armazenamento

Agora que já temos os dados prontos para manipulação, podemos usar o `pyspark` para inserir os dados em nosso banco de dados relacional da Oracle. Para isto, vamos usar usar o [driver JDBC da Oracle](https://www.oracle.com/br/database/technologies/appdev/jdbc-downloads.html). *Esta etapa vai falhar se o driver __não__ estiver instalado no cluster atual*, para efetuar a instalação, seguimos os passos indicados [neste vídeo](https://youtu.be/3tAVXfIBqA8?si=jNeO459775ag9x44&t=261).

Vamos começar definindo um `schema` para todos os nomes de colunas possíveis. Para decidir qual o *data type* ideal para cada coluna, usamos o [dicionário de dados](https://dadosabertos.ans.gov.br/FTP/PDA/TISS/DICIONARIO/Dicionario_de_variaveis.ods) fornecido pela ANS.

In [ ]:
VAR_TYPES = StructType()\
    .add("TEMPO_DE_PERMANENCIA", IntegerType(), True)\
    .add("ID_EVENTO_ATENCAO_SAUDE", IntegerType(), True)\
    .add("CD_TABELA_REFERENCIA", StringType(), True)\
    .add("CD_MUNICIPIO_BENEFICIARIO", StringType(), True)\
    .add("CD_MUNICIPIO_PRESTADOR", StringType(), True)\
    .add("UF_PRESTADOR", StringType(), True)\
    .add("CD_CARATER_ATENDIMENTO", StringType(), True)\
    .add("CD_TIPO_INTERNACAO", StringType(), True)\
    .add("CD_REGIME_INTERNACAO", StringType(), True)\
    .add("CD_MOTIVO_SAIDA", StringType(), True)\
    .add("IND_ACIDENTE_DOENCA", StringType(), True)\
    .add("ANO_MES_EVENTO", StringType(), True)\
    .add("CD_PROCEDIMENTO", StringType(), True)\
    .add("CID_1", StringType(), True)\
    .add("CID_2", StringType(), True)\
    .add("CID_3", StringType(), True)\
    .add("CID_4", StringType(), True)


### Variáveis de conexão com o banco de dados
Agora vamos obter as variáveis que vão nos ajudar a conectar à *"autonomous database"* que já temos alocada na Oracle.

- A `TNS_STR` é providenciada pela OCI como uma opção de conexão ao banco de dados;
- `DRIVER` é o driver de conexão que o `PySpark` vai usar para conectar com o banco de dados da Oracle;
- Vamos usar uma string de conexão `URL`, que usa informações contidas na `TNS_STR`, além do usuário `USR` e senha ` PWD` 

In [ ]:
WALLET_PLACE = input("Insira o caminho completo para a pasta da sua wallet: ")
JARS_PLACE = input("Insira o caminho completo para a pasta que contém os arqivos .jar necessários: ")


conf = SparkConf()
conf.set("spark.jars", path.join(JARS_PLACE, "*"))
drivername = "oracle.jdbc.OracleDriver"
URL = f"jdbc:oracle:thin:@demodb_high?TNS_ADMIN={WALLET_PLACE}"
USR = "ADMIN"
PWD = getpass("Insira a senha de administrador do banco de dados: ")

spark = SparkSession.builder\
	.config(conf=conf)\
	.getOrCreate()

Agora que temos o `schema` e as **instruções de conexão** prontos, podemos usá-los para ler os dados que obtemos interpretanto os tipos corretamente. Abaixo podemos ver uma amostra das tabelas consolidadas:

In [ ]:
cons_df = s.read.format("csv")\
    .option("header", True)\
    .option("delimiter", ";")\
    .option("schema", VAR_TYPES)\
    .csv("dbfs:/ans/hosp/2019/CONS/")

cons_df.show(5)

ler depois
- https://docs.oracle.com/cloud/help/pt_BR/analytics-cloud/ACSDS/GUID-FB2AEC3B-2178-48DF-8B9F-76ED2D6B5194.htm#ACSDS-GUID-FB2AEC3B-2178-48DF-8B9F-76ED2D6B5194
- https://docs.oracle.com/en-us/iaas/autonomous-database-serverless/doc/connect-jdbc-thin-wallet.html#GUID-BE543CFD-6FB4-4C5B-A2EA-9638EC30900D

In [ ]:
tablename = "hosp_cons_2019"

cons_df.write.format("jdbc")\
	.option("driver", drivername)\
	.option("url", URL)\
	.option("dbtable", tablename )\
	.option("user", USR)\
	.option("password", PWD)\
	.save()

In [ ]:
det_df = s.read.format("csv")\
    .option("header", True)\
    .option("delimiter", ";")\
    .option("schema", VAR_TYPES)\
    .csv("dbfs:/ans/hosp/2019/DET/")

det_df.show()

In [ ]:
drivername = "oracle.jdbc.OracleDriver"
tablename = "hosp_det_2019"

det_df.write.format("jdbc")\
	.option("driver", drivername)\
	.option("url", URL)\
	.option("dbtable", tablename )\
	.option("user", USR)\
	.option("password", PWD)\
	.save()